In [2]:

#Area calc -> CRS:6875

# from osgeo import gdal, ogr, osr
import numpy as np
import cv2

import rasterio
import geopandas as gpd
from rasterio.features import shapes
from shapely.geometry import shape, box

from owslib.wms import WebMapService
import pandas as pd

from PIL import Image
import io

In [3]:

# min_lat = 44.5084337
# min_lon = 11.3444083
# max_lat = 44.5090871
# max_lon = 11.3456463

min_lat = 42.2005615
min_lon = 13.6668152
max_lat = 42.2008607
max_lon = 13.6670757

# Definizione dei colori
color_particella = np.array([189, 236, 253])  # CP.CadastralParcel
color_fabbricato = np.array([19, 128, 236])  # Fabbricati

# Percorso dell'immagine
path_img_input = r"test_shape_post_algo\raster.png"
output_shapefile = r'IO\Output\shape_estratte.shp'


## Codice per WMS

In [6]:


def ScaleUpBox(df):
 
  scale_value = 0.00003

  df[["min_lat_scaled", "min_lon_scaled"]] = df[["min_lat", "min_lon"]] - scale_value
  df[["max_lat_scaled", "max_lon_scaled"]] = df[["max_lat", "max_lon"]] + scale_value
  
  # return df

def ExtractShapeFromRow(row):
  def GetShapesFromWMS(row):

    try:
      img =  wms.getmap(layers=["CP.CadastralParcel", "fabbricati"],
                        srs="EPSG:6706",
                        bbox=(row["min_lat_scaled"], row["min_lon_scaled"], row["max_lat_scaled"], row["max_lon_scaled"]),
                        size=(2048, 2048),
                        format="image/png",
                        transparent=True
                        )

      out = open(path_img_input, "wb")
      out.write(img.read())
      # img.show()
    except:
      raise ValueError("Errore nella chiamata al WMS")

  def ComputeRaster(row):
    # Carica l'immagine con OpenCV (BGR format)
    img = cv2.imread(path_img_input)

    # Crea maschere per i due colori
    mask_fabbricato = cv2.inRange(img, color_fabbricato, color_fabbricato)
    mask_particella = cv2.inRange(img, color_particella, color_particella)

    # Carica il raster con rasterio
    with rasterio.open(path_img_input) as src:
        # Dimensioni dell'immagine
        # height, width, _ = img.shape

        # Trasformazione tra bounding box e pixel
        transform = rasterio.transform.from_bounds(row["min_lon_scaled"], row["min_lat_scaled"], row["max_lon_scaled"], row["max_lat_scaled"], 2048, 2048)

        # Polygonizza la maschera arancione
        shapes_fabbricato = shapes(mask_fabbricato, mask=mask_fabbricato > 0, transform=transform)
        
        # Polygonizza la maschera gialla
        shapes_particella = shapes(mask_particella, mask=mask_particella > 0, transform=transform)

        # Converti entrambe le maschere in GeoDataFrame e assegna le proprietà "Fabbricato" e "Particella"
        geoms_fabbricati = [{'geometry': shape(geom), 'properties': {'Tipo': 'Fabbricato'}} for geom, value in shapes_fabbricato]
        geoms_particelle = [{'geometry': shape(geom), 'properties': {'Tipo': 'Particella'}} for geom, value in shapes_particella]
        
        # Combina i geodataframe
        gdf = gpd.GeoDataFrame.from_features(geoms_fabbricati + geoms_particelle, crs=6706)

    # ** Elimina le shape vicine ai bordi del bbox **
    # Crea il bounding box e riducilo usando la soglia di distanza
    bbox = box(row["min_lon_scaled"], row["min_lat_scaled"], row["max_lon_scaled"], row["max_lat_scaled"])
    bbox_distance_threshold = 0.00002
    bbox_restricted = bbox.buffer(-bbox_distance_threshold)

    # Filtra le geometrie all'interno del bbox ridotto
    gdf = gdf[gdf.geometry.within(bbox_restricted)]

    #SMOOTHING
    tolerance = 0.000001 # Imposta una tolleranza adeguata
    gdf['geometry'] = gdf['geometry'].simplify(tolerance, preserve_topology=True)

    # Salva il risultato come shapefile
    gdf.to_file(output_shapefile)

  GetShapesFromWMS(row)
  ComputeRaster(row)


# MAIN


wms = WebMapService('https://wms.cartografia.agenziaentrate.gov.it/inspire/wms/ows01.php?', version='1.3.0')

data = {
    'id': [1],
    'min_lat': min_lat,
    'min_lon': min_lon,
    'max_lat': max_lat,
    'max_lon': max_lon,
}
df = pd.DataFrame(data)

df = ScaleUpBox(df)

df.apply(ExtractShapeFromRow, axis=1)


print("end")


# https://wms.cartografia.agenziaentrate.gov.it/inspire/wms/ows01.php?SERVICE=WMS&VERSION=1.3.0&REQUEST=GetMap&FORMAT=image%2Fpng&TRANSPARENT=true&LAYERS=CP.CadastralParcel,fabbricati&CRS=EPSG%3A6706&STYLES=&BBOX=42.2005615%2C13.6668152%2C42.2008607%2C13.6670757&WIDTH=1783&HEIGHT=2048

#FAQ

#https://wms.cartografia.agenziaentrate.gov.it/inspire/wms/ows01.php?SERVICE=WMS&VERSION=1.3.0&REQUEST=GetMap&BBOX=41.89666254472871287,12.47067418651901782,41.90105270407683236,12.47615685111776251&CRS=EPSG:4258&WIDTH=1089&HEIGHT=872&LAYERS=CP.CadastralZoning,strade,acque,CP.CadastralParcel,fabbricati,vestizioni&STYLES=default&FORMAT=image/png&DPI=96&MAP_RESOLUTION=96&FORMAT_OPTIONS=dpi:96&TRANSPARENT=TRUE

In [ ]:
# Carica l'immagine con OpenCV (BGR format)
img = cv2.imread(path_img_input)

# Crea maschere per i due colori
mask_fabbricato = cv2.inRange(img, color_fabbricato, color_fabbricato)
mask_particella = cv2.inRange(img, color_particella, color_particella)

# Carica il raster con rasterio
with rasterio.open(path_img_input) as src:
    # Dimensioni dell'immagine
    height, width, _ = img.shape

    # Trasformazione tra bounding box e pixel
    transform = rasterio.transform.from_bounds(min_lon, min_lat, max_lon, max_lat, width, height)

    # Polygonizza la maschera arancione
    shapes_fabbricato = shapes(mask_fabbricato, mask=mask_fabbricato > 0, transform=transform)
    
    # Polygonizza la maschera gialla
    shapes_particella = shapes(mask_particella, mask=mask_particella > 0, transform=transform)

    # Converti entrambe le maschere in GeoDataFrame e assegna le proprietà "Fabbricato" e "Particella"
    geoms_fabbricati = [{'geometry': shape(geom), 'properties': {'Tipo': 'Fabbricato'}} for geom, value in shapes_fabbricato]
    geoms_particelle = [{'geometry': shape(geom), 'properties': {'Tipo': 'Particella'}} for geom, value in shapes_particella]
    
    # Combina i geodataframe
    gdf = gpd.GeoDataFrame.from_features(geoms_fabbricati + geoms_particelle, crs=6706)

# ** Elimina le shape vicine ai bordi del bbox **
# Crea il bounding box e riducilo usando la soglia di distanza
bbox = box(min_lon, min_lat, max_lon, max_lat)
bbox_distance_threshold = 0.00001
bbox_restricted = bbox.buffer(-bbox_distance_threshold)

# Filtra le geometrie all'interno del bbox ridotto
gdf = gdf[gdf.geometry.within(bbox_restricted)]

# ** Smoothing delle shape (semplificazione) **
# Applica uno smoothing usando la funzione simplify di Shapely (aggiusta la tolleranza come necessario)
tolerance = 0.000001  # Imposta una tolleranza adeguata
gdf['geometry'] = gdf['geometry'].simplify(tolerance, preserve_topology=True)

# Salva il risultato come shapefile
gdf.to_file(output_shapefile)


In [ ]:
#TODO -> le shape nere non mi servono, ne contorni ne altro
#TODO -> smoothing magari -> da fare poi


def polygonize_mask(mask, geotransform, projection, output_shapefile):
    # Crea un dataset in memoria per la maschera
    driver = gdal.GetDriverByName('MEM')
    mask_ds = driver.Create('', mask.shape[1], mask.shape[0], 1, gdal.GDT_Byte)
    mask_ds.GetRasterBand(1).WriteArray(mask)
    
    # Imposta il geotransform e il CRS
    mask_ds.SetGeoTransform(geotransform)
    mask_ds.SetProjection(projection)
    
    # Polygonizza
    output_ds = ogr.GetDriverByName('ESRI Shapefile').CreateDataSource(output_shapefile)
    srs = osr.SpatialReference()
    srs.ImportFromWkt(projection)
    layer = output_ds.CreateLayer('', srs, ogr.wkbPolygon)
    gdal.Polygonize(mask_ds.GetRasterBand(1), None, layer, -1, [], callback=None)

# Percorso dell'immagine
path_img_input = fr"raster.png"

# Carica l'immagine con OpenCV
img = cv2.imread(path_img_input)

height, width, _ = img.shape

# Calcola le dimensioni dei pixel in coordinate geografiche
pixel_width = (max_lon - min_lon) / width
pixel_height = (max_lat - min_lat) / height

geotransform = [min_lon, pixel_width, 0, max_lat, 0, -pixel_height]
image_ds = gdal.Open(path_img_input)
projection = image_ds.GetProjection()

# Definisci i colori in BGR
color_1 = np.array([189, 236, 253])  # CP.CadastralParcel
color_2 = np.array([19, 128, 236])  # Fabbricati

# Crea maschere per i colori specifici
mask1 = cv2.inRange(img, color_1, color_1)
mask2 = cv2.inRange(img, color_2, color_2)

# Combina le maschere
combined_mask = cv2.bitwise_or(mask1, mask2)

# Salva o visualizza la maschera per verifica (opzionale)
# cv2.imshow('Combined Mask', combined_mask)
# cv2.imwrite('\test_shape_post_algo\combined_mask.png', combined_mask)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# Polygonizza la maschera combinata
output_shapefile = r'test_shape_post_algo\combined_shapes.shp'
polygonize_mask(combined_mask, geotransform, projection, output_shapefile)